In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
#from keras.layers import Dense, Embedding, LSTM, SimpleRNN, SpatialDropout1D
from keras.layers import Dense, Activation, Flatten
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, Conv1D, GlobalMaxPool1D

In [ ]:
data = pd.read_csv("/content/FinalDatasetmerged (1).csv")
data.head(2)

,content,sentiment_labels
0,feature search hotspot location able function ...,-1.0
1,good,2.0


In [ ]:
data=data.dropna()
data.shape

(32183, 2)

In [ ]:
!pip install keras.utils 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2656 sha256=65812f146d515777f24df90ef15d91b3749fcfa354b9f9bf50f1a998a96ae028
  Stored in directory: /root/.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f
Successfully built keras.utils


In [ ]:
data["sentiment_labels"].replace({-1: 0}, inplace=True)
data['sentiment_labels'].unique()

array([0., 2., 1.])

In [ ]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(data['sentiment_labels'], num_classes=3)
labels.shape
#labels = data['label'].values
#labels

(32183, 3)

In [ ]:
max_features = 15000 #Max Num Words
max_len = 300 #Max Input Length #100
token = Tokenizer(num_words=max_features)
token.fit_on_texts(data['content'].values)
sequences = token.texts_to_sequences(data['content'].values)
X = pad_sequences(sequences, maxlen=max_len)#.reshape(-1,1)
word_index = token.word_index
print('Found %s unique tokens.' % len(word_index))

Found 13961 unique tokens.


In [ ]:
y = labels#.reshape(-1,1)
y.shape,X.shape

((32183, 3), (32183, 300))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [ ]:
def build_model(embed_dim=50, optimizer='adam', spatial_dropout=0.3, lstm_dropout=0.25): #lstm_units=128
  ls_model = Sequential()
  ls_model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
  #ls_model.add(SpatialDropout1D(spatial_dropout))
  ls_model.add((LSTM(embed_dim, dropout=lstm_dropout, recurrent_dropout=lstm_dropout,return_sequences=True)))
  ls_model.add(Flatten())
  ls_model.add(Dense(3, activation='softmax'))# Dense layers improve overall accuracy and 5–10 units or nodes per layer is a good base.
  ls_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])
  ls_model.summary()
  return ls_model

In [ ]:
keras_estimator = KerasClassifier(build_fn=build_model, verbose=1)

# define the randomized search parameters
param_grid = {
    'epochs': [5],         # 10, 15],
    'embed_dim':[50,100],
    'optimizer':['adam','RMSprop'],
    'spatial_dropout': [0.2,0.3],
    'lstm_dropout': [0.20,0.25],
    #'lstm_units': [64,128,256],
    'batch_size':[128,64],
}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


In [12]:
from sklearn.model_selection import RandomizedSearchCV
kfold_splits = 2
from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score, average='weighted')
random_search = RandomizedSearchCV(estimator=keras_estimator,  
                          #n_jobs=-1, 
                          verbose=2,
                          refit=False,
                          scoring=f1,
                          cv=kfold_splits,
                          error_score='raise',
                          return_train_score=True,
                          param_distributions=param_grid)

tuning_results = random_search.fit(X_train, np.argmax(y_train, axis=1),) 


# summarize results
print("Best: %f using %s" % (tuning_results.best_score_, tuning_results.best_params_))
means = tuning_results.cv_results_['mean_test_score']
stds = tuning_results.cv_results_['std_test_score']
params = tuning_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          1500000   
                                                                 
 lstm (LSTM)                 (None, 300, 100)          80400     
                                                                 
 flatten (Flatten)           (None, 30000)             0         
                                                                 
 dense (Dense)               (None, 3)                 90003     
                                                                 
Total params: 1,670,403
Trainable params: 1,670,403
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
101/101 [==============================] - 144s 1s/step - loss: 0.7697 - acc: 0.6733
Epoch 2/5
101/101 [===

In [13]:
tuning_results.best_params_

{'batch_size': 128,
 'embed_dim': 100,
 'epochs': 5,
 'lstm_dropout': 0.2,
 'optimizer': 'adam',
 'spatial_dropout': 0.3}

In [ ]:
final_model = build_model(embed_dim=embed_dim, optimizer='adam', spatial_dropout=0.4, lstm_dropout=0.3, lstm_units=128)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
final_model.fit(X_train, y_train, epochs=35, batch_size=64, class_weight=None,
                callbacks= [ModelCheckpoint('model.hdf5', monitor='loss', 
                                            verbose=1, save_best_only=True)], 
                verbose=2)

In [ ]:
loss, accuracy = final_model.evaluate(X_test, y_test)
print("loss", loss)
print("accuracy", accuracy)

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
pred = ls_model.predict(X_test)
labels = ['Negative', 'Neutral', 'Positive']
y_pred_labels = [labels[x] for x in [np.argmax(x) for x in pred]]
y_test_labels = [labels[x] for x in [np.argmax(x) for x in y_test]]
confusion = confusion_matrix(y_test_labels, y_pred_labels)
confusion_df = pd.DataFrame(confusion,index=[labels], columns=[labels])
print(confusion_df)